In [237]:
# import necessary libraries

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision
from torchvision import transforms
from torchvision.models import resnet18
from torchvision import utils, datasets
from torchvision.datasets import Cityscapes, MNIST
from torchsummary import summary

from torch.utils.data import Dataset, DataLoader

import os
import sys
import numpy as np
import matplotlib
import time
import tqdm

from PIL import Image

# for experiment tracking
import neptune
import neptune_pytorch
from getpass import getpass

state_dict_only =  1
whole_model = 2
both = 0

# check gpu availablity

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [238]:
os.chdir('/home/zkim/drivablearea/semantic-segmentation/')

In [239]:
#import parameters 
import yaml

cfg = yaml.safe_load(open("./configs/cityscapes.yaml"))
dataset_cfg, model_cfg = cfg['DATASET'], cfg['MODEL']
train_cfg, eval_cfg, test_cfg = cfg['TRAIN'], cfg['EVAL'], cfg['TEST']
#model = eval(model_cfg['NAME'])(model_cfg['BACKBONE'], trainset.n_classes)


In [240]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
image_size = cfg['TRAIN']['IMAGE_SIZE']
batch_size = cfg['TRAIN']['BATCH_SIZE']
epoch = cfg['TRAIN']['EPOCHS']
learning_rate = 0.001
weight_decay = 0.01

In [241]:
#experiment tracking tool 

run = neptune.init_run(
    project="zkim/drivable-area",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI4NDRiNjgzYi1lNGRiLTQzY2YtYjUwMy1mYTFjOWI0YzM1NTIifQ==",
) 

parameters = {
    "lr": 1e-2,
    "bs": 128,
    "input_sz": 32 * 32 * 3,
    "n_classes": 10,
    "model_filename": "basemodel",
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "epochs": 2,
}
# if you need to use it
# model = Model(
#     parameters["input_sz"], parameters["input_sz"], parameters["n_classes"]
# ).to(parameters["device"])
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=parameters["lr"])

/tmp/ipykernel_401037/3393632705.py:3: NeptuneWarning: The following monitoring options are disabled by default in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', and 'capture_hardware_metrics'. To enable them, set each parameter to 'True' when initializing the run. The monitoring will continue until you call run.stop() or the kernel stops. Also note: Your source files can only be tracked if you pass the path(s) to the 'source_code' argument. For help, see the Neptune docs: https://docs.neptune.ai/logging/source_code/
  run = neptune.init_run(


https://app.neptune.ai/zkim/drivable-area/e/DRIV-1


In [188]:
dataset_cfg['ROOT']

'data/CityScapes'

In [351]:
# use torchvision.datasets to process data


dataset_path = dataset_cfg['ROOT']

transforms_train_img = transforms.Compose([
    #transforms.PILToTensor(),
    #transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    # transforms.ColorJitter(brightness=0.0, contrast=0.5, saturation=0.5, hue=0.5),
    # transforms.RandomAdjustSharpness(sharpness_factor=0.1, p=0.5),
    # transforms.RandomHorizontalFlip(p=0.5),
    # transforms.RandomVerticalFlip(p=0.5),
    # transforms.RandomGrayscale(p=0.5),
    # transforms.RandomRotation(degrees=10, p=0.3, seg_fill=seg_fill),
    # transforms.RandomResizedCrop(size, scale=(0.5, 2.0), seg_fill=seg_fill),
    transforms.Resize(size=(256,512)), # default size: [2048 w, 1024 h]
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])
transforms_train_label = transforms.Compose([
    # transforms.ColorJitter(brightness=0.0, contrast=0.5, saturation=0.5, hue=0.5),
    # transforms.RandomAdjustSharpness(sharpness_factor=0.1, p=0.5),
    # transforms.RandomHorizontalFlip(p=0.5),
    # transforms.RandomVerticalFlip(p=0.5),
    # transforms.RandomGrayscale(p=0.5),
    # transforms.RandomRotation(degrees=10, p=0.3, seg_fill=seg_fill),
    # transforms.RandomResizedCrop(size, scale=(0.5, 2.0), seg_fill=seg_fill),
    transforms.Resize(size=(256,512)), # default size: [2048 w, 1024 h]
    transforms.ToTensor(),
])

transforms_val_test = transforms.Compose([
    #transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    transforms.Resize(size=(256,512)),
    transforms.ToTensor(),
])

# cityscapes_train_dataset = Cityscapes(dataset_path, split='train', mode='fine', target_type='semantic', transform=transforms.ToTensor(), target_transform=transforms.ToTensor())
cityscapes_train_dataset = Cityscapes(dataset_path, split='train', mode='fine', target_type='semantic', transform=transforms_train_img, target_transform=transforms_train_label)
cityscapes_val_dataset = Cityscapes(dataset_path, split='val', mode='fine', target_type='semantic', transform=transforms_val_test, target_transform=transforms_val_test)
cityscapes_test_dataset = Cityscapes(dataset_path, split='test', mode='fine', target_type='semantic', transform=transforms_val_test, target_transform=transforms_val_test)


In [344]:
img, label = cityscapes_train_dataset[0]

In [348]:
img, smnt = cityscapes_test_dataset[1]
type(smnt)


torch.Tensor

In [346]:
torchvision.transforms.functional.get_image_size(img)


[512, 256]

In [352]:
#dataloader 

#from semseg.augmentations import *
#from semseg.datasets import CamVid

train_dataloader = DataLoader(cityscapes_train_dataset, batch_size=batch_size, shuffle="False", num_workers=4)
val_dataloader = DataLoader(cityscapes_val_dataset, batch_size=batch_size, shuffle="False", num_workers=4)
test_dataloader = DataLoader(cityscapes_test_dataset, batch_size=batch_size, shuffle="False", num_workers=4)

In [353]:
print(train_dataloader)

In [258]:
from semseg import show_models, show_backbones, show_heads, show_datasets

show_backbones()

Backbone Names    Variants
----------------  ------------------------------------
ResNet            ['18', '34', '50', '101', '152']
ResNetD           ['18', '50', '101']
MicroNet          ['M1', 'M2', 'M3']
MobileNetV2       ['1.0']
MobileNetV3       ['S', 'L']
MiT               ['B0', 'B1', 'B2', 'B3', 'B4', 'B5']
PVTv2             ['B1', 'B2', 'B3', 'B4', 'B5']
ResT              ['S', 'B', 'L']
PoolFormer        ['S24', 'S36', 'M36']
ConvNeXt          ['T', 'S', 'B']


In [ ]:
# class CustomCNN(BaseModel):
#     def __init__(self, backbone: str = 'ResNet-50', num_classes: int = 19): backbone in string
#         super().__init__(backbone, num_classes)
#         self.decode_head = UPerHead(self.backbone.channels, 256, num_classes) # choose head
#         self.apply(self._init_weights)

#     def forward(self, x: Tensor) -> Tensor:
#         y = self.backbone(x)
#         y = self.decode_head(y)   # 4x reduction in image size
#         y = F.interpolate(y, size=x.shape[2:], mode='bilinear', align_corners=False)    # to original image shape
#         return y

    
# if __name__ == '__main__':
#     model = CustomCNN('ResNet-18', 19)
#     model.init_pretrained('checkpoints/backbones/resnet/resnet18.pth')
#     x = torch.randn(2, 3, 224, 224)
#     y = model(x)
#     print(y.shape)

In [13]:
# save and load pretrained model

def load_model(model, model_path, save_what):   
    if (save_what == state_dict_only):
        model.load_state_dict(torch.load(model_path))
        model.to(device)

    if (save_what == whole_model):
        model = torch.load(model_path)
        model.to(device)


def save_model(model, save_path, save_what):
    if (save_what == state_dict_only or both):
        torch.save(model.state_dict(), save_path)

    if (save_what == whole_model or both):
        torch.save(save_path)


# model_path = '/home/zkim/outdoor_navigation/semantic-segmentation/checkpoints/pretrained/ddrnet/DDRNet23s_imagenet.pth'  # change if necessary
# model = DDRNet().to(device) #   or some other neural network with pretrained state dict
# model.load_state_dict(torch.load(model_path))


In [26]:
import torch 
import time
import multiprocessing as mp
from tabulate import tabulate
from tqdm import tqdm
from pathlib import Path
from torch.utils.tensorboard import SummaryWriter
from torch.cuda.amp import GradScaler, autocast
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DistributedSampler, RandomSampler
from torch import distributed as dist
from semseg.models import *
from semseg.datasets import * 
from semseg.augmentations import get_train_augmentation, get_val_augmentation
from semseg.losses import get_loss
from semseg.schedulers import get_scheduler
from semseg.optimizers import get_optimizer
from semseg.utils.utils import fix_seeds, setup_cudnn, cleanup_ddp, setup_ddp
from tools.val import evaluate

In [354]:
best_mIoU = 0.0
num_workers = 4 #mp.cpu_count() = 24 in this case
train_cfg, eval_cfg = cfg['TRAIN'], cfg['EVAL']
dataset_cfg, model_cfg = cfg['DATASET'], cfg['MODEL']
loss_cfg, optim_cfg, sched_cfg = cfg['LOSS'], cfg['OPTIMIZER'], cfg['SCHEDULER']
epochs, lr = epoch, optim_cfg['LR']

# transforms
# traintransform = get_train_augmentation(train_cfg['IMAGE_SIZE'], seg_fill=dataset_cfg['IGNORE_LABEL'])
# valtransform = get_val_augmentation(eval_cfg['IMAGE_SIZE'])

trainset = cityscapes_train_dataset
valset = cityscapes_val_dataset 

#img, smnt = cityscapes_train_dataset[0]

In [355]:
# model = eval(model_cfg['NAME'])(model_cfg['BACKBONE'], trainset.n_classes)
# model.init_pretrained(model_cfg['PRETRAINED'])
model = torchvision.models.resnet18(weights='DEFAULT') # temporary model to set to make the pipeline
num_classes = 19
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)
model = model.to(device)


if train_cfg['DDP']: 
    sampler = DistributedSampler(trainset, dist.get_world_size(), dist.get_rank(), shuffle=True)
    model = DDP(model, device_ids=[gpu])
else:
    sampler = RandomSampler(trainset)

trainloader = train_dataloader
valloader = val_dataloader    


In [231]:
len(trainset)

2975

In [356]:
iters_per_epoch = len(trainset) // batch_size
# class_weights = trainset.class_weights.to(device)
loss_function = nn.CrossEntropyLoss() #get_loss(loss_cfg['NAME'], trainset.ignore_label, None)
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay) #get_optimizer(model, optim_cfg['NAME'], lr, optim_cfg['WEIGHT_DECAY'])
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1) #get_scheduler(sched_cfg['NAME'], optimizer, epochs * iters_per_epoch, sched_cfg['POWER'], iters_per_epoch * sched_cfg['WARMUP'], sched_cfg['WARMUP_RATIO'])
scaler = GradScaler(enabled=train_cfg['AMP'])
#writer = SummaryWriter(str(save_dir / 'logs'))


In [252]:
# train

def train_model(model, trainloader, loss_fn, optimizer, scheduler, epoch, lr):
    start = time.time()

    for epoch in range(epochs):
        model.train()
        if train_cfg['DDP']: sampler.set_epoch(epoch)

        train_loss = 0.0
        pbar = tqdm.tqdm(enumerate(trainloader), total=iters_per_epoch, desc=f"Epoch: [{epoch+1}/{epochs}] Iter: [{0}/{iters_per_epoch}] LR: {lr:.8f} Loss: {train_loss:.8f}")

        for iter, (img, lbl) in pbar:
            optimizer.zero_grad(set_to_none=True)

            img = img.to(device)
            lbl = lbl.to(device)
            print(type(img))
            print(type(lbl))
            
            with autocast(enabled=train_cfg['AMP']):
                logits = model(img)
                loss = loss_fn(logits, lbl)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            torch.cuda.synchronize()

            lr = scheduler.get_lr()
            lr = sum(lr) / len(lr)
            train_loss += loss.item()

            pbar.set_description(f"Epoch: [{epoch+1}/{epochs}] Iter: [{iter+1}/{iters_per_epoch}] LR: {lr:.8f} Loss: {train_loss / (iter+1):.8f}")
        
        train_loss /= iter+1
        #writer.add_scalar('train/loss', train_loss, epoch)
        torch.cuda.empty_cache()

        if (epoch+1) % train_cfg['EVAL_INTERVAL'] == 0 or (epoch+1) == epochs:
            miou = evaluate(model, valloader, device)[-1]
            #writer.add_scalar('val/mIoU', miou, epoch)

            if miou > best_mIoU:
                best_mIoU = miou
                torch.save(model.module.state_dict() if train_cfg['DDP'] else model.state_dict(), save_dir / f"{model_cfg['NAME']}_{model_cfg['BACKBONE']}_{dataset_cfg['NAME']}.pth")
            print(f"Current mIoU: {miou} Best mIoU: {best_mIoU}")

    #writer.close()
    pbar.close()
    end = time.gmtime(time.time() - start)

    table = [
        ['Best mIoU', f"{best_mIoU:.2f}"],
        ['Total Training Time', time.strftime("%H:%M:%S", end)]
    ]
    print(tabulate(table, numalign='right'))    
    end = time.time()
    print("time taken in s: %lf", (end - start))
    


In [253]:
train_model(model, trainloader, loss_function, optimizer, scheduler, epoch, learning_rate)

Epoch: [1/500] Iter: [0/1487] LR: 0.00100000 Loss: 0.00000000:   0%|          | 0/1487 [00:00<?, ?it/s]

<class 'torch.Tensor'>
<class 'torch.Tensor'>


OutOfMemoryError: CUDA out of memory. Tried to allocate 256.00 MiB. GPU 0 has a total capacty of 5.79 GiB of which 128.00 MiB is free. Process 346816 has 66.00 MiB memory in use. Process 348036 has 66.00 MiB memory in use. Process 350007 has 66.00 MiB memory in use. Including non-PyTorch memory, this process has 4.78 GiB memory in use. Of the allocated memory 4.67 GiB is allocated by PyTorch, and 31.66 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [298]:
#print(cityscapes_val_dataset.CityscapesClass)
#val_dataloader.dataset.classes[1][6]
#dataloader.dataset.ignore_label
len(val_dataloader.dataset.classes)

35

### Validation

In [398]:
import torch
import math
from pathlib import Path
from tqdm import tqdm
from tabulate import tabulate
from torch.utils.data import DataLoader
from torch.nn import functional as F
from semseg.models import *
from semseg.datasets import *
from semseg.augmentations import get_val_augmentation
from semseg.metrics import Metrics
from semseg.utils.utils import setup_cudnn

def evaluate(model, dataloader, device):
    #@torch.no_grad()
    
    print('Evaluating...')
    model.eval()
    #metrics = Metrics(dataloader.dataset.n_classes, dataloader.dataset.ignore_label, device)
    metrics = Metrics(len(val_dataloader.dataset.classes), dataloader.dataset.classes[0][6], device)
    num_classes = len(val_dataloader.dataset.classes)
    hist = torch.zeros(num_classes, num_classes).to(device)
    
    for images, labels in tqdm(dataloader):
        images = images.to(device)
        labels = labels.to(device)
        preds = model(images).softmax(dim=1) 
        print('preds.size(): ', preds.size())  #torch.Size([2, 1000])
        print('labels.size(): ', labels.size()) #torch.Size([2, 1, 256, 512])
        #metrics.update(preds, labels)
        keep = labels != dataloader.dataset.classes[0][6]
        print(keep)
        preds = preds.argmax(dim=1)
        #keep = labels != dataloader.dataset.classes[0][6]
        print('preds: ', preds)
        print('preds.size(): ', preds.size())
        print('preds[keep].size(): ', preds[keep].size())
        print('labels[keep]: ', labels[keep])
        print('labels[keep].size(): ', labels[keep].size())
        hist += torch.bincount(labels[keep] * num_classes + preds[keep], minlength=num_classes**2).view(num_classes, num_classes)
    
    ious, miou = metrics.compute_iou()
    acc, macc = metrics.compute_pixel_acc()
    f1, mf1 = metrics.compute_f1()
    
    return acc, macc, f1, mf1, ious, miou


#dataloader - val_loader



In [399]:
# evaluation 


device = torch.device('cpu')

eval_cfg = cfg['EVAL']
dataset = cityscapes_val_dataset
dataloader = val_dataloader

model_path = '' #Path(eval_cfg['MODEL_PATH'])
# if not model_path.exists(): 
#         model_path = Path(cfg['SAVE_DIR']) / f"{cfg['MODEL']['NAME']}_{cfg['MODEL']['BACKBONE']}_{cfg['DATASET']['NAME']}.pth"
print(f"Evaluating {model_path}...")

model = resnet18() # some model
# original - model = eval(cfg['MODEL']['NAME'])(cfg['MODEL']['BACKBONE'], dataset.n_classes)
#model.load_state_dict(torch.load(model_path, map_location='cpu'))
model = model.to(device)

# if eval_cfg['MSF']['ENABLE']:
#     acc, macc, f1, mf1, ious, miou = evaluate_msf(model, dataloader, device, eval_cfg['MSF']['SCALES'], eval_cfg['MSF']['FLIP'])
# else:
acc, macc, f1, mf1, ious, miou = evaluate(model, dataloader, device)

table = {
    'Class': list(dataset.CLASSES) + ['Mean'],
    'IoU': ious + [miou],
    'F1': f1 + [mf1],
    'Acc': acc + [macc]
}

print(tabulate(table, headers='keys'))

#def test_model(model, trainloader, loss_fn, optimizer, scheduler, epoch, lr):

Evaluating ...
Evaluating...


  0%|          | 0/250 [00:01<?, ?it/s]

preds.size():  torch.Size([2, 1000])
labels.size():  torch.Size([2, 1, 256, 512])
tensor([[[[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True]]],


        [[[True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          ...,
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True],
          [True, True, True,  ..., True, True, True]]]])
preds:  tensor([909, 909])
preds.size():  torch.Size([2])


IndexError: too many indices for tensor of dimension 1

In [377]:
def evaluate_msf(model, dataloader, device, scales, flip): # use multiscale and flip to 
    torch.no_grad()
    model.eval()

    n_classes = len(val_dataloader.dataset.classes) #dataloader.dataset.n_classes
    metrics = Metrics(n_classes, dataloader.dataset.ignore_label, device)

    for images, labels in tqdm(dataloader):
        labels = labels.to(device)
        B, H, W = labels.shape
        scaled_logits = torch.zeros(B, n_classes, H, W).to(device)

        for scale in scales:
            new_H, new_W = int(scale * H), int(scale * W)
            new_H, new_W = int(math.ceil(new_H / 32)) * 32, int(math.ceil(new_W / 32)) * 32
            scaled_images = F.interpolate(images, size=(new_H, new_W), mode='bilinear', align_corners=True)
            scaled_images = scaled_images.to(device)
            logits = model(scaled_images)
            logits = F.interpolate(logits, size=(H, W), mode='bilinear', align_corners=True)
            scaled_logits += logits.softmax(dim=1)

            if flip:
                scaled_images = torch.flip(scaled_images, dims=(3,))
                logits = model(scaled_images)
                logits = torch.flip(logits, dims=(3,))
                logits = F.interpolate(logits, size=(H, W), mode='bilinear', align_corners=True)
                scaled_logits += logits.softmax(dim=1)

        metrics.update(scaled_logits, labels)
    
    acc, macc = metrics.compute_pixel_acc()
    f1, mf1 = metrics.compute_f1()
    ious, miou = metrics.compute_iou()
    return acc, macc, f1, mf1, ious, miou